In [1]:
import time
import psycopg2
import chardet as cd
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

with open("contract_info.csv",'rb') as temp:
    encodingType=cd.detect(temp.read(100000))
    
data=pd.read_csv("contract_info.csv",encoding=encodingType['encoding'])
db_engine  = create_engine('postgresql+psycopg2://' + 'postgres' + ':' + 'wyh638' + '@localhost' + ':' + str(5432) + '/' + 'Project1_python')

orders_f=data[['contract number','product model','quantity','estimated delivery date','lodgement date','salesman number']].copy()
orders_f['lodgement date']=pd.to_datetime(data['lodgement date'],format='%Y/%m/%d')
orders_f['estimated delivery date']=pd.to_datetime(data['estimated delivery date'],format='%Y/%m/%d')
orders_f.loc[orders_f['lodgement date']>pd.to_datetime('2022-03-02'),['lodgement date']]=None
data_t=np.arange(1,len(orders_f)+1).reshape(len(orders_f),-1)
data_f=pd.DataFrame(data_t,columns=['orders_number'])
orders=pd.concat([data_f,orders_f],axis=1)
orders_reSet=orders
orders_del=orders
orders_add=orders
#orders.info()
#orders.to_csv('orders.csv',index=False)

contract=data[['contract number','client enterprise','supply center','contract date']].copy()
contract.drop_duplicates(subset='contract number',inplace=True)
#contract.info()

product=data[['product model','product code','product name','unit price']].copy()
product.drop_duplicates(subset='product model',inplace=True)
#product.info()

salesman=data[['salesman number','salesman','gender','age','mobile phone']].copy()
salesman.drop_duplicates(subset='salesman number',inplace=True)
salesman.drop_duplicates(subset='mobile phone',inplace=True)
#salesman.info()

In [70]:
#无条件，distinct
time_start=time.time()
databaseData = pd.read_sql("select distinct(salemans_id) from orders",con=db_engine)      
time_end=time.time()
print('DBMS totally cost',(time_end-time_start)*1000, ' ms')

time_start=time.time()
orders.drop_duplicates(subset='salesman number',inplace=False)['salesman number']
time_end=time.time()
print('Python totally cost',(time_end-time_start)*1000, ' ms')


DBMS totally cost 13.557195663452148  ms
Python totally cost 3.004789352416992  ms


In [50]:
#单条件查询
time_start=time.time()
databaseData = pd.read_sql("select * from orders where quantity=480",con=db_engine)      
time_end=time.time()
print('DBMS totally cost',(time_end-time_start)*1000, ' ms')

time_start=time.time()
orders[((orders.quantity==480))]
time_end=time.time()
print('Python totally cost',(time_end-time_start)*1000, ' ms')

DBMS totally cost 10.5133056640625  ms
Python totally cost 2.1996498107910156  ms


In [49]:
#多条件查询
time_start=time.time()
databaseData = pd.read_sql("select * from orders where quantity=480 and estimated_d>('2022-01-01'::date)",con=db_engine)      
time_end=time.time()
print('DBMS totally cost',(time_end-time_start)*1000, ' ms')


time_start=time.time()
orders[((orders.quantity==480)|(orders['estimated delivery date']>pd.to_datetime('2022-01-01')))]
time_end=time.time()
print('Python totally cost',(time_end-time_start)*1000, ' ms')

DBMS totally cost 12.992620468139648  ms
Python totally cost 3.005504608154297  ms


In [89]:
#聚合函数
time_start=time.time()
databaseData = pd.read_sql("select salemans_id,max(quantity) from orders group by (salemans_id)",con=db_engine)
time_end=time.time()
print('DBMS totally cost',(time_end-time_start)*1000, ' ms')

time_start=time.time()
t=orders.groupby('salesman number').max('quantity')
time_end=time.time()
print('Python totally cost',(time_end-time_start)*1000, ' ms')

DBMS totally cost 16.190767288208008  ms
Python totally cost 5.995988845825195  ms


In [123]:
#多表查询inner join
time_start=time.time()
databaseData = pd.read_sql("select contract_number,product_m,p.name,p.code from orders join product p on orders.product_m=p.model;",con=db_engine)
time_end=time.time()
print('inner join DBMS totally cost',(time_end-time_start)*1000, ' ms')

time_start=time.time()
result=pd.merge(orders,product,on='product model')[['contract number','product model','product name','product code']]
time_end=time.time()
print('inner join Python totally cost',(time_end-time_start)*1000, ' ms')

#多表查询 leftjoin
time_start=time.time()
databaseData = pd.read_sql("select contract_number,product_m,p.name,p.code from orders left join product p on orders.product_m=p.model;",con=db_engine)
time_end=time.time()
print('left join DBMS totally cost',(time_end-time_start)*1000, ' ms')

time_start=time.time()
result=pd.merge(orders,product,on='product model',how='left')[['contract number','product model','product name','product code']]
time_end=time.time()
print('left join Python totally cost',(time_end-time_start)*1000, ' ms')

inner join DBMS totally cost 156.99458122253418  ms
inner join Python totally cost 18.998146057128906  ms
left join DBMS totally cost 137.0067596435547  ms
left join Python totally cost 15.601158142089844  ms


In [91]:
#多表查询inner join（3个表）
time_start=time.time()
databaseData = pd.read_sql("select contract_number,product_m,p.name,p.code,s.name from orders join product p on orders.product_m=p.model join salesman s on orders.salemans_id = s.number;",con=db_engine)
time_end=time.time()
print('inner join（3） DBMS totally cost',(time_end-time_start)*1000, ' ms')

time_start=time.time()
result=pd.merge(orders,product,on='product model')
result=pd.merge(result,salesman,on='salesman number')[['contract number','product model','product name','product code','salesman']]
time_end=time.time()
print('left join(3) Python totally cost',(time_end-time_start)*1000, ' ms')

inner join（3） DBMS totally cost 233.98756980895996  ms
left join(3) Python totally cost 46.99993133544922  ms


In [111]:
#更新数据
time_start=time.time()
pd.read_sql("update orders set quantity =quantity*2 where contract_number='CSE0004999'",con=db_engine,chunksize=1000)
time_end=time.time()
print('DBMS1 totally cost',(time_end-time_start)*1000, ' ms')

time_start=time.time()
orders_reSet.loc[orders_reSet['contract number']=='CSE0004999',['quantity']]*=2
time_end=time.time()
print('Python totally cost',(time_end-time_start)*1000, ' ms')

DBMS1 totally cost 14.067411422729492  ms
Python totally cost 5.010843276977539  ms


In [124]:
#删除数据（删36条数据）
time_start=time.time()
pd.read_sql("delete from orders where contract_number='CSE0004999'",con=db_engine,chunksize=1)
time_end=time.time()
print('DBMS totally cost',(time_end-time_start)*1000, ' ms')

time_start=time.time()
orders_del.drop(orders_del[orders_del['contract number']=='CSE0004999'].index,inplace=True)
time_end=time.time()
print('Python totally cost',(time_end-time_start)*1000, ' ms')

DBMS totally cost 15.002965927124023  ms
Python totally cost 9.990215301513672  ms


In [2]:
#增添数据
time_start=time.time()
for i in range(1,10000):
    pd.read_sql("insert into orders values("+str(50000+i)+",'CSE0004999','TvBaseR1',1,('2022-01-01'::date),('2022-01-01'::date),11211429);",con=db_engine,chunksize=1000)
    #print("insert into orders values("+str(50000+i)+",'CSE0004999','TvBaseR1',1,('2022-01-01'::date),('2022-01-01'::date),11211429");
time_end=time.time()
print('DBMS totally cost',(time_end-time_start)*1000, ' ms')

time_start=time.time()
for i in range(1,10000):
    orders_add.loc[orders_add.shape[0]]=[orders_add.shape[0]+1,'CSE0004999','TvBaseR1',1,pd.to_datetime('2022-01-01'),pd.to_datetime('2022-01-01'),11211429]
time_end=time.time()
print('Python totally cost',(time_end-time_start)*1000, ' ms')

DBMS totally cost 16913.057804107666  ms
Python totally cost 61468.04356575012  ms
